In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, SimpleRNN, BatchNormalization
from keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from nltk.stem import ISRIStemmer
import string, re, emoji, Stemmer
import pyarabic.araby as ar
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import accuracy_score, classification_report

In [2]:
file_path = "train.xlsx"
df_train = pd.read_excel(file_path)
df_train

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1
...,...,...
32031,التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...,-1
32032,y love you,1
32033,الباقه بتخلص وبشحن مرتين باقه اضافيه ١٠٠ جنيه,-1
32034,تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجة...,-1


In [3]:
arabic_stop_words = [
    "و", "في", "من", "على", "إلى", "لا", "أو", "هو", "هي", "يكون",
    "أنا", "أنت", "هو", "هي", "نحن", "أنتم", "هم",
    "عن", "مع", "كما", "مثل", "بين", "إذا", "حتى", "منذ",
    "و", "أو", "لكن", "إذا", "إن",
    "اليوم", "غداً", "الآن", "ثم", "بعد",
    "كان", "يكون", "أصبح", "صار", "ليس", "لم",
    "هذا", "هذه", "ذلك", "تلك", 
    "كل", "على", "فيه", "منه", "عنه", "له", "به", "إليه", "لها", "فيها",
    "بها", "منها", "عنها", "إليها", "الذي", "التي", "اللذين", "اللذان", "اللتان",
    "اللتين", "هؤلاء", "ذلك", "هذه", "هذا", "تلك", "تحت", "فوق", "معه", "لديه",
    "عليه", "عليها", "أي", "هل", "إذا", "ماذا", "هناك", "هنالك", "إلى",
    "يناير", "فبراير", "مارس", "إبريل", "مايو", "يونيو", "يوليو", "أغسطس", "سبتمبر", "أكتوبر", "نوفمبر", "ديسمبر",
    "الأحد", "الاثنين", "الثلاثاء", "الأربعاء", "الخميس", "الجمعة", "السبت"
]

In [4]:
st = Stemmer.Stemmer("arabic")


def text_cleaning(text, stemmer=st):
    # remove stop words and punctuation

    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub("(\s\d+)", "", text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text = re.sub("\d+", " ", text)
    text = ar.strip_tashkeel(text)
    text = ar.strip_tatweel(text)
    text = text.replace("#", " ")
    text = text.replace("@", " ")
    text = text.replace("_", " ")

    tokenizer = RegexpTokenizer(r"\w+")

    words = tokenizer.tokenize(text)

    stop_words = set(string.punctuation).union(set(arabic_stop_words))

    filtered_list = [word for word in words if word.casefold() not in stop_words]
    
    # word stemming
    stem_words = [stemmer.stemWord(word) for word in filtered_list]

    text = " ".join(map(str, stem_words))
    text = text.replace("آ", "ا")
    text = text.replace("إ", "ا")
    text = text.replace("أ", "ا")
    text = text.replace("ؤ", "و")
    text = text.replace("ئ", "ي")

    return text



df_train["new review_description"] = df_train["review_description"].apply(
    lambda text: text_cleaning(text)
)



df_train.head(20)

,review_description,rating,new review_description
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1,شرك زبال سواق تبرشم مفيش حت رقم شكاو سواق يسيب...
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1,خدم دفع طريق الك نت توقف عند اصبح فقط دفع نقد
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1,تطبيق غب جار حذف عامل اكواد خصم لما استخدم اكت...
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1,فعل تطبيق ممتاز بس لو في امكان يتيح لمستخدم تط...
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1,سيء جدا اسعار رسوم توصيل تمت واقع ب صله
5,قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...,0,قعد عشرين سنة يدور سايق بس اما توصيل اشياء جيد...
6,احلئ تطبيق,1,احلء تطبيق
7,رائع واو مدهش,1,رايع واو مدهش
8,مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...,-1,مکو بس بحری عمان غیر بس عراق مکو یعنی نجم وحد ...
9,تطبيق جميل يستاهل الخمس نجوم👍👍👍,1,تطبيق جميل استاهل خمس نجوم


In [5]:
X = df_train["new review_description"]
y = df_train["rating"].astype(int) + 1

In [6]:
tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(X)
total_words = len(tokenizer.word_index) + 1

In [7]:
sequences = tokenizer.texts_to_sequences(X)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, y, test_size=0.2, random_state=42, stratify=y
)
num_classes = 3

In [32]:
model = Sequential()
model.add(
    Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=300,
        input_length=X_train.shape[1],
    )
)
model.add(LSTM(32, activation="tanh", dropout=0.5))
model.add(Dense(num_classes, activation="softmax", kernel_regularizer=l2(0.1)))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint = ModelCheckpoint(
    filepath="./models_hdf5/LSTM-emb.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [33]:
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint],
)

Epoch 1/5
801/801 [==============================] - 32s 37ms/step - loss: 0.6814 - accuracy: 0.8015 - val_loss: 0.5230 - val_accuracy: 0.8411
Epoch 2/5
801/801 [==============================] - 29s 37ms/step - loss: 0.4480 - accuracy: 0.8660 - val_loss: 0.5081 - val_accuracy: 0.8349
Epoch 3/5
801/801 [==============================] - 29s 37ms/step - loss: 0.3763 - accuracy: 0.8936 - val_loss: 0.5212 - val_accuracy: 0.8336
Epoch 4/5
801/801 [==============================] - 30s 37ms/step - loss: 0.3329 - accuracy: 0.9057 - val_loss: 0.5474 - val_accuracy: 0.8268
Epoch 5/5
801/801 [==============================] - 31s 38ms/step - loss: 0.3084 - accuracy: 0.9116 - val_loss: 0.5581 - val_accuracy: 0.8257


In [34]:
model.load_weights("./models_hdf5/LSTM-emb.hdf5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

201/201 [==============================] - 3s 15ms/step - loss: 0.5230 - accuracy: 0.8411
Validation Loss: 0.5230498909950256, Validation Accuracy: 0.84113609790802


In [16]:
model2 = Sequential()
model2.add(
    Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=300,
        input_length=X_train.shape[1],
    )
)
model2.add(SimpleRNN(16, activation="tanh", dropout=0.5))
model2.add(Dense(num_classes, activation="softmax"))
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint = ModelCheckpoint(
    filepath="./models_hdf5/RNN-emb.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [17]:
model2.fit(
    X_train,
    y_train,
    epochs=2,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint],
)

Epoch 1/2
801/801 [==============================] - 413s 513ms/step - loss: 0.6011 - accuracy: 0.7731 - val_loss: 0.5622 - val_accuracy: 0.7929
Epoch 2/2
801/801 [==============================] - 373s 466ms/step - loss: 0.4766 - accuracy: 0.8340 - val_loss: 0.5751 - val_accuracy: 0.7993


In [26]:
model2.load_weights("./models_hdf5/RNN-emb.hdf5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

201/201 [==============================] - 3s 14ms/step - loss: 0.4639 - accuracy: 0.8366
Validation Loss: 0.4638712406158447, Validation Accuracy: 0.8366104960441589


# TESTING

In [35]:
file_path = "test _no_label.csv"
df_test = pd.read_csv(file_path)
df_test

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت
...,...,...
995,996,يستهل
996,997,خدمة سيئة بكل المعايير
997,998,لؤي٠٣٣٢لؤ٣٤٣س
998,999,تطبيق غير صادق ف خصم الكوبونات


In [36]:
df_test["review_description"] = df_test["review_description"].apply(
    lambda text: text_cleaning(text)
)
X_test_seq = tokenizer.texts_to_sequences(df_test["review_description"])
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [37]:
y_pred_lstm = model.predict(X_test_pad)
y_pred_rnn = model2.predict(X_test_pad)

32/32 [==============================] - 2s 51ms/step


In [38]:
def return_prediction(y_pred):
    prediction = []
    for predict in y_pred:
        maxi = np.argmax(predict)
        if maxi == 2:
            prediction.append(1)
        elif maxi == 1:
            prediction.append(0)
        else:
            prediction.append(-1)
    return prediction


prediction_lstm = return_prediction(y_pred_lstm)
prediction_rnn = return_prediction(y_pred_rnn)

print(len(prediction_lstm), len(prediction_rnn))

1000 1000


In [39]:
results_df = pd.DataFrame({"ID": df_test["ID"], "rating": prediction_lstm})
results_df.to_csv("./results/test_results_LSTM_emb.csv", index=False)
results_df = pd.DataFrame({"ID": df_test["ID"], "rating": prediction_rnn})
results_df.to_csv("./results/test_results_rnn_emb.csv", index=False)

In [40]:
comparison_result = np.array(prediction_lstm) == np.array(prediction_rnn)
not_equal_idx = np.where(comparison_result != True)
print("The equal values are : ", np.sum(comparison_result))

The equal values are :  884


In [41]:
for idx in not_equal_idx[0]:
    text = df_test.iloc[idx, 1]
    rnn_prediction = prediction_rnn[idx]
    lstm_prediction = prediction_lstm[idx]
    print(f"Text: {text}, RNN: {rnn_prediction}, LSTM: {lstm_prediction}")

Text: ماجرب, RNN: -1, LSTM: 1
Text: ء, RNN: -1, LSTM: 1
Text: انا طلب وجب ببس وصل ساع سبع, RNN: 1, LSTM: -1
Text: خالص تقدير فكره بجد ذات شركه تنفذ ينبغ قدر امك تحيات رجاء تحسين جود تتبع رحله, RNN: 1, LSTM: -1
Text: ياجماع انا عمر ما طلب تليف اطلب الخط مفيش اي عروض, RNN: 1, LSTM: -1
Text: اطلب يتز بيج ماك مشهور تامر عرب خلوه دي منور دنيا ينزل ا, RNN: -1, LSTM: 1
Text: سسس تيشيلغاليسسششسبهصسسشششششسسصسسيسسسسسسششبسشسنتخليشيسسششسسسشششضضحض حذاء كرت كرت كرت عبد وهاب علاج فير حساب عبد وهاب علاج فير, RNN: -1, LSTM: 1
Text: يعجب, RNN: -1, LSTM: 1
Text: انه يحدد موقع فيوم, RNN: -1, LSTM: 1
Text: جميل جدا ولا يسال اسال كثير, RNN: -1, LSTM: 1
Text: رايع بس ليش ماف مدين خميس مشيط, RNN: -1, LSTM: 1
Text: خدم عملاء ذوق جدا, RNN: 1, LSTM: -1
Text: عملاء قحين, RNN: 1, LSTM: -1
Text: اسمع يوصل قر, RNN: -1, LSTM: 1
Text: انا موجود قطر برو سيت مسيمير ولا استطيع استخدام تطبيق لطلب طعام دايم يرسل موقع خارج نطاق خدم, RNN: 1, LSTM: -1
Text: تاخير بعض مطاعم وصول اكل بارد, RNN: 1, LSTM: -1
Text: تحديث, RNN: -1,